In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [2]:
import tensorflow as tf
from sklearn.metrics import classification_report

2024-03-14 17:46:42.157334: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 17:46:42.157354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 17:46:42.157842: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
seed = 42
dataset_path = '/home/deeplearning/AI/datasets/TrashNet'
epochs = 50
batch_size = 32
lr = 0.01
weight_decay = None

In [4]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=seed),
    tf.keras.layers.RandomRotation(factor=0.05, seed=seed),
    tf.keras.layers.RandomContrast(factor=0.2, seed=seed)
])

In [5]:
train_ds_raw = tf.keras.utils.image_dataset_from_directory(dataset_path + '/train', batch_size=batch_size)
train_ds = train_ds_raw.map(lambda img, label: (data_augmentation(img), label)).shuffle(64, seed=seed).prefetch(tf.data.AUTOTUNE)

Found 1768 files belonging to 6 classes.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/val', batch_size=batch_size)

Found 328 files belonging to 6 classes.


In [7]:
class_names = train_ds_raw.class_names
num_classes = len(class_names)

In [8]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B2(
    weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.momentum = 0.90

In [10]:
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[model_checkpoint_callback])

Epoch 1/50
56/56 [==============================] - 8s 56ms/step - loss: 0.0064 - accuracy: 0.9972 - val_loss: 0.1862 - val_accuracy: 0.9512
Epoch 2/50
56/56 [==============================] - 8s 57ms/step - loss: 0.0095 - accuracy: 0.9966 - val_loss: 0.2008 - val_accuracy: 0.9390
Epoch 3/50
56/56 [==============================] - 8s 56ms/step - loss: 0.0117 - accuracy: 0.9966 - val_loss: 0.2056 - val_accuracy: 0.9482
Epoch 4/50
56/56 [==============================] - 7s 54ms/step - loss: 0.0086 - accuracy: 0.9972 - val_loss: 0.1799 - val_accuracy: 0.9573
Epoch 5/50
56/56 [==============================] - 7s 55ms/step - loss: 0.0068 - accuracy: 0.9977 - val_loss: 0.1877 - val_accuracy: 0.9512
Epoch 6/50
56/56 [==============================] - 8s 56ms/step - loss: 0.0061 - accuracy: 0.9994 - val_loss: 0.1919 - val_accuracy: 0.9482
Epoch 7/50
56/56 [==============================] - 8s 56ms/step - loss: 0.0094 - accuracy: 0.9977 - val_loss: 0.2116 - val_accuracy: 0.9421
Epoch 8/50
56

In [8]:
saved_model_path = 'models/efficientnet_v2_trash_net.keras' #model_checkpoint_callback.filepath
saved_model = tf.keras.models.load_model(saved_model_path)
saved_model

In [9]:
saved_model.evaluate(val_ds)

11/11 [==============================] - 2s 33ms/step - loss: 0.1568 - accuracy: 0.9634


[0.15681712329387665, 0.9634146094322205]

In [10]:
test_ds = tf.keras.utils.image_dataset_from_directory(dataset_path + '/test', batch_size=batch_size)

Found 431 files belonging to 6 classes.


In [11]:
saved_model.evaluate(test_ds)

14/14 [==============================] - 0s 32ms/step - loss: 0.2906 - accuracy: 0.9397


[0.2906416952610016, 0.9396751523017883]

In [12]:
labels = []
predictions = []

for X, y in test_ds:
    y_pred = tf.argmax(saved_model.predict(X, verbose=0), axis=1).numpy().tolist()
    labels.extend(y.numpy().tolist())
    predictions.extend(y_pred)

In [13]:
print(classification_report(labels, predictions, target_names=test_ds.class_names))

              precision    recall  f1-score   support

   cardboard       0.97      0.93      0.95        70
       glass       0.95      0.90      0.93        82
       metal       0.93      0.97      0.95        68
       paper       0.95      0.96      0.95       108
     plastic       0.91      0.95      0.93        74
       trash       0.93      0.90      0.91        29

    accuracy                           0.94       431
   macro avg       0.94      0.93      0.94       431
weighted avg       0.94      0.94      0.94       431

